In [9]:
import keras
from matplotlib import pyplot as plt
import numpy as np
import h5py
import tensorflow as tf

In [10]:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [11]:
keras.backend.image_data_format()

'channels_first'

In [12]:
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [13]:
x_train = x_train / 255.
x_test = x_test / 255.

In [14]:
x_train.shape

(60000, 28, 28)

In [15]:
x_train = x_train.reshape(-1, 1, 28, 28)
x_test = x_test.reshape(-1, 1, 28, 28)

In [16]:
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten
from keras.models import Sequential

In [17]:
import keras.backend.tensorflow_backend as tfback

In [18]:
def _get_available_gpus():  

    if tfback._LOCAL_DEVICES is None:  
        devices = tf.config.list_logical_devices()  
        tfback._LOCAL_DEVICES = [x.name for x in devices]  
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]


tfback._get_available_gpus = _get_available_gpus

In [19]:
lenet = Sequential()
lenet.add(Conv2D(6, kernel_size=3, strides=1, padding='same', input_shape=(1, 28, 28), activation="relu", \
                name='conv1', data_format="channels_first"))

lenet.add(MaxPool2D(pool_size=2, strides=2, name='pool1', data_format='channels_first'))

lenet.add(Conv2D(16, kernel_size=5, strides=1, padding='valid', activation="relu", \
                 name='conv2', data_format="channels_first"))

lenet.add(MaxPool2D(pool_size=2, strides=2, name='pool2', data_format="channels_first"))
lenet.add(Flatten(name='flatten', data_format="channels_last"))
lenet.add(Dense(120, activation="relu", name='fc1'))
lenet.add(Dense(84, activation="relu", name='fc2'))
lenet.add(Dense(10, activation='softmax', name='softmax'))

In [20]:
lenet.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 6, 28, 28)         60        
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 6, 14, 14)         0         
_________________________________________________________________
conv2 (Conv2D)               (None, 16, 10, 10)        2416      
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 16, 5, 5)          0         
_________________________________________________________________
flatten (Flatten)            (None, 400)               0         
_________________________________________________________________
fc1 (Dense)                  (None, 120)               48120     
_________________________________________________________________
fc2 (Dense)                  (None, 84)               

In [22]:
# lenet.compile('sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
# lenet.fit(x_train, y_train, batch_size=64, epochs=50, validation_data=[x_test, y_test])

In [24]:
# lenet.save('LeNet_NCHW.h5')

In [166]:
lenet.load_weights("LeNet_NCHW2.h5")

In [167]:
image_7 = np.expand_dims(x_test[0], axis=0)
predictions = lenet.predict(image_7)

In [168]:
predictions

array([[4.1750793e-13, 1.0217871e-07, 4.7025295e-09, 7.9027460e-09,
        4.6409793e-11, 4.0204432e-14, 3.6066829e-20, 9.9999988e-01,
        5.8345616e-12, 4.1460571e-10]], dtype=float32)

In [169]:
from keras.models import Model

In [170]:
with open("conv1_in.dat","w") as f:
    for i in image_7[0]:
        for j in i:
            for k in j:
                f.write(str(k)+"\n")

In [171]:
m_conv_out = Model(inputs=lenet.input, outputs=lenet.get_layer('conv1').output)
Y_conv_out = m_conv_out.predict(image_7)
print(Y_conv_out.shape)

(1, 6, 28, 28)


In [172]:
with open("conv1_out_relu.dat","w") as f:
    for i in Y_conv_out[0]:
        for j in i:
            for k in j:
                f.write(str(k)+"\n")

In [206]:
f = h5py.File("LeNet_NCHW2.h5","r")

In [207]:
list(f.keys())

['model_weights', 'optimizer_weights']

In [208]:
f['model_weights'].keys()

<KeysViewHDF5 ['conv1', 'conv2', 'fc1', 'fc2', 'flatten', 'pool1', 'pool2', 'softmax']>

In [209]:
conv1_kernel=f['model_weights']['conv1']['conv1']['kernel:0'][:]

In [210]:
conv1_kernel.shape

(3, 3, 1, 6)

In [211]:
conv1_kernel= np.moveaxis(conv1_kernel,-1,0)
conv1_kernel= np.moveaxis(conv1_kernel,-1,1)
conv1_kernel.shape

(6, 1, 3, 3)

In [212]:
with open("conv1_kernel.dat","w") as f:
    for i in conv1_kernel:
        for j in i[0]:
            for k in j:
                f.write(str(k)+"\n")

In [180]:
f = h5py.File("LeNet_NCHW2.h5","r")

In [181]:
conv1_bias = f['model_weights']['conv1']['conv1']['bias:0'][:]

In [182]:
conv1_bias.shape

(6,)

In [183]:
with open("conv1_bias.dat","w") as f:
    for i in conv1_bias:
        f.write(str(i)+"\n")

In [184]:
# with open("conv1_kernel.dat","w") as f:
#     for i in conv1_kernel[0]:
#         for j in i:
#             for k in j:
#                 f.write(str(k)+"\n")

In [185]:
# with open("conv1_bias.dat","w") as f:
#     for i in conv1_bias:
#         f.write(str(i)+"\n")

In [186]:
conv_out = np.zeros(shape=(6,28,28))
buf=np.zeros(shape=(28,28))
for o in range(6):
    
    for r in range(28):
        for c in range(28):
            buf[r][c]=conv1_bias[o]
    
    for i in range(1):
        for r in range(28):
            for c in range(28):
                acc=0
                for kr in range(3):
                    for kc in range(3):
                        
                        ridx = r + kr - int(3/2)
                        cidx = c + kc - int(3/2)
                        if(ridx < 0 or ridx >= 28 or cidx < 0 or cidx >= 28):
                            data = 0;
                        else:
                            data=image_7[0][i][ridx][cidx];
                        
                        acc+=conv1_kernel[o][i][kr][kc]*data
                    
                buf[r][c]+=acc
                
    for r in range(28):
        for c in range(28):
            conv_out[o][r][c] = buf[r][c]

In [163]:
# conv_out = conv_out.clip(min=0)

In [187]:
conv_out.shape

(6, 28, 28)

In [188]:
with open("conv1_out.dat","w") as f:
    for i in conv_out:
        for j in i:
            for k in j:
                f.write(str(k)+"\n")

In [189]:
conv_out = conv_out.clip(min=0)

In [190]:
conv_out.shape

(6, 28, 28)

In [191]:
Y_conv_out[0].shape

(6, 28, 28)

In [200]:
idx = np.where((Y_conv_out[0] == conv_out)==False)

In [203]:
idx[0].shape

(1065,)

In [204]:
conv1_kernel.shape

(6, 1, 3, 3)